In [57]:
import os
import sys
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score, classification_report

In [58]:
stock_symbol= 'ADANIPORTS'

In [59]:
def dataset_collecion(stock_symbol):
    price_data = pd.read_csv('../data/processed/stocks/nse_scraped/'+stock_symbol+".csv")
    return price_data


In [60]:
price_data=dataset_collecion(stock_symbol)
price_data

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.00,962.90,984.72,27294366
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.00,893.90,941.38,4581338
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.00,884.20,888.09,5124121
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.00,921.55,929.17,4609762
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.00,969.30,965.65,2977470
...,...,...,...,...,...,...,...,...,...,...,...
3547,2022-03-28,ADANIPORTS,EQ,743.25,747.00,749.70,726.55,735.90,737.00,735.00,5691095
3548,2022-03-29,ADANIPORTS,EQ,737.00,741.00,766.90,738.05,760.25,761.95,754.87,11423551
3549,2022-03-30,ADANIPORTS,EQ,761.95,768.80,775.95,762.10,765.50,767.40,769.15,6913983
3550,2022-03-31,ADANIPORTS,EQ,767.40,767.40,781.00,765.50,772.65,774.20,774.64,5468951


In [61]:
def values_sort_price_change_calculation(price_data):
    price_data = price_data[['Date','Symbol','Prev Close','Open',
                             'High','Low','Last','Close','VWAP','Volume']]

    # sort the values by symbol and then date
    price_data.sort_values(by = ['Symbol','Date'], inplace = True)

    # calculate the change in price
    price_data['change_in_price'] = price_data['Close'].diff()
    return price_data


In [62]:
new_price_data = values_sort_price_change_calculation(price_data)


d:\anaconda\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [63]:
new_price_data

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,change_in_price
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,NaN
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,1.70
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,7.70
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,6.00
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,-8.65
...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,4.25
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,0.60
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,-4.40
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,-1.10


In [66]:
def row_symbol_change(new_price_data):
    # identify rows where the symbol changes
    mask = new_price_data['Symbol'] != new_price_data['Symbol'].shift(1)

    # For those rows, let's make the value null
    new_price_data['change_in_price'] = np.where(mask == True, np.nan, new_price_data['change_in_price'])

    # print the rows that have a null value, should only be 5
    new_price_data[new_price_data.isna().any(axis = 1)]
    return new_price_data

In [67]:
row_symbol_change(new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,change_in_price
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,NaN
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,1.70
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,7.70
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,6.00
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,-8.65
...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,4.25
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,0.60
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,-4.40
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,-1.10


In [68]:
# define the number of days out you want to predict
days_out = 30
def grouping_signal_flag(days_out,new_price_data):
    # Group by symbol, then apply the rolling function and grab the Min and Max.
    new_price_data_smoothed = new_price_data.groupby(['Symbol'])[['Close','Low','High','Open','Volume']].transform(lambda x: x.ewm(span = days_out).mean())

    # Join the smoothed columns with the symbol and datetime column from the old data frame.
    smoothed_df = pd.concat([new_price_data[['Symbol','Date']], new_price_data_smoothed], axis=1, sort=False)

    return smoothed_df

In [71]:
smoothed_df=grouping_signal_flag(days_out,new_price_data)

In [72]:
def signal_flag(days_out,smoothed_df):
# define the number of days out you want to predict
    #days_out = 30

    # create a new column that will house the flag, and for each group calculate the diff compared to 30 days ago. Then use Numpy to define the sign.
    smoothed_df['Signal_Flag'] = smoothed_df.groupby('Symbol')['Close'].transform(lambda x : np.sign(x.diff(days_out)))

    # print the first 50 rows
    return smoothed_df

In [73]:
signal_flag(days_out,smoothed_df)

,Symbol,Date,Close,Low,High,Open,Volume,Signal_Flag
1023,ADANIPORTS,2012-01-17,140.000000,135.000000,141.000000,137.100000,1.636196e+06,NaN
1024,ADANIPORTS,2012-01-18,140.878333,136.911667,142.446667,139.631667,1.250967e+06,NaN
1025,ADANIPORTS,2012-01-19,143.910292,139.131229,145.329785,141.185894,1.323944e+06,NaN
1026,ADANIPORTS,2012-01-20,147.076126,142.194856,148.710677,144.138021,1.409395e+06,NaN
1027,ADANIPORTS,2012-01-23,147.001924,142.855846,150.232660,146.700394,1.465870e+06,NaN
...,...,...,...,...,...,...,...,...
1018,MUNDRAPORT,2012-01-10,128.102974,124.811614,130.807004,128.137302,1.182316e+06,-1.0
1019,MUNDRAPORT,2012-01-11,128.680201,125.452800,131.445262,128.644573,1.222860e+06,-1.0
1020,MUNDRAPORT,2012-01-12,128.936317,125.823587,131.835890,129.215891,1.256791e+06,-1.0
1021,MUNDRAPORT,2012-01-13,129.104942,126.064001,131.910994,129.427769,1.246657e+06,-1.0


In [77]:
# Calculate the 14 day RSI
n = 14
def RSI(n,new_price_data):
    # First make a copy of the data frame twice
    up_df, down_df = new_price_data[['Symbol','change_in_price']].copy(), new_price_data[['Symbol','change_in_price']].copy()

    # For up days, if the change is less than 0 set to 0.
    up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

    # For down days, if the change is greater than 0 set to 0.
    down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

    # We need change in price to be absolute.
    down_df['change_in_price'] = down_df['change_in_price'].abs()

    # Calculate the EWMA (Exponential Weighted Moving Average), meaning older values are given less weight compared to newer values.
    ewma_up = up_df.groupby('Symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
    ewma_down = down_df.groupby('Symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())

    # Calculate the Relative Strength
    relative_strength = ewma_up / ewma_down

    # Calculate the Relative Strength Index
    relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

    # Add the info to the data frame.
    new_price_data['down_days'] = down_df['change_in_price']
    new_price_data['up_days'] = up_df['change_in_price']
    new_price_data['RSI'] = relative_strength_index

    # Display the head.
    return new_price_data

In [78]:
RSI(n,new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,change_in_price,down_days,up_days,RSI
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,NaN,NaN,NaN,NaN
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,1.70,0.00,1.70,100.000000
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,7.70,0.00,7.70,100.000000
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,6.00,0.00,6.00,100.000000
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,-8.65,8.65,0.00,58.293541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,4.25,0.00,4.25,73.223117
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,0.60,0.00,0.60,73.976523
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,-4.40,4.40,0.00,59.751127
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,-1.10,1.10,0.00,56.610918


In [79]:
# Calculate the Stochastic Oscillator
n = 14
def Stochastic_Oscillator(n,new_price_data):
    # Make a copy of the high and low column.
    low_14,high_14 = new_price_data[['Symbol','Low']].copy(), new_price_data[['Symbol','High']].copy()

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    low_14 = low_14.groupby('Symbol')['Low'].transform(lambda x: x.rolling(window = n).min())
    high_14 = high_14.groupby('Symbol')['High'].transform(lambda x: x.rolling(window = n).max())

    # Calculate the Stochastic Oscillator.
    k_percent = 100 * ((new_price_data['Close'] - low_14) / (high_14 - low_14))

    # Add the info to the data frame.
    new_price_data['low_14'] = low_14
    new_price_data['high_14'] = high_14
    new_price_data['k_percent'] = k_percent
    return new_price_data

In [80]:
Stochastic_Oscillator(n,new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,1.70,0.00,1.70,100.000000,NaN,NaN,NaN
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,7.70,0.00,7.70,100.000000,NaN,NaN,NaN
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,6.00,0.00,6.00,100.000000,NaN,NaN,NaN
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,-8.65,8.65,0.00,58.293541,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,4.25,0.00,4.25,73.223117,111.0,137.0,97.884615
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,0.60,0.00,0.60,73.976523,111.0,140.7,87.710438
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,-4.40,4.40,0.00,59.751127,111.0,140.7,72.895623
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,-1.10,1.10,0.00,56.610918,111.0,140.7,69.191919


In [81]:
# Calculate the Williams %R
n = 14
def Williams (n,new_price_data): 
    # Make a copy of the high and low column.
    low_14, high_14 = new_price_data[['Symbol','Low']].copy(), new_price_data[['Symbol','High']].copy()

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    low_14 = low_14.groupby('Symbol')['Low'].transform(lambda x: x.rolling(window = n).min())
    high_14 = high_14.groupby('Symbol')['High'].transform(lambda x: x.rolling(window = n).max())

    # Calculate William %R indicator.
    r_percent = ((high_14 - new_price_data['Close']) / (high_14 - low_14)) * - 100

    # Add the info to the data frame.
    new_price_data['r_percent'] = r_percent

    # Display the head.
    return new_price_data

In [82]:
Williams (n,new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,1.70,0.00,1.70,100.000000,NaN,NaN,NaN,NaN
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,7.70,0.00,7.70,100.000000,NaN,NaN,NaN,NaN
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,6.00,0.00,6.00,100.000000,NaN,NaN,NaN,NaN
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,-8.65,8.65,0.00,58.293541,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,4.25,0.00,4.25,73.223117,111.0,137.0,97.884615,-2.115385
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,0.60,0.00,0.60,73.976523,111.0,140.7,87.710438,-12.289562
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,-4.40,4.40,0.00,59.751127,111.0,140.7,72.895623,-27.104377
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,-1.10,1.10,0.00,56.610918,111.0,140.7,69.191919,-30.808081


In [83]:
def MACD(n,new_price_data):
    # Calculate the MACD
    ema_26 = new_price_data.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span = 26).mean())
    ema_12 = new_price_data.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span = 12).mean())
    macd = ema_12 - ema_26

    # Calculate the EMA
    ema_9_macd = macd.ewm(span = 9).mean()

    # Store the data in the data frame.
    new_price_data['MACD'] = macd
    new_price_data['MACD_EMA'] = ema_9_macd

    # Print the head.
    return new_price_data

In [84]:
MACD(n,new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,1.70,0.00,1.70,100.000000,NaN,NaN,NaN,NaN,0.038141,0.021189
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,7.70,0.00,7.70,100.000000,NaN,NaN,NaN,NaN,0.287376,0.130282
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,6.00,0.00,6.00,100.000000,NaN,NaN,NaN,NaN,0.610317,0.292896
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,-8.65,8.65,0.00,58.293541,NaN,NaN,NaN,NaN,0.428747,0.333308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,4.25,0.00,4.25,73.223117,111.0,137.0,97.884615,-2.115385,-0.298741,-2.412779
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,0.60,0.00,0.60,73.976523,111.0,140.7,87.710438,-12.289562,0.514220,-1.827379
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,-4.40,4.40,0.00,59.751127,111.0,140.7,72.895623,-27.104377,0.794298,-1.303044
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,-1.10,1.10,0.00,56.610918,111.0,140.7,69.191919,-30.808081,0.916931,-0.859049


In [85]:
# Calculate the Price Rate of Change
w = 9
def rate_of_change(w,new_price_data):
    # Calculate the Rate of Change in the Price, and store it in the Data Frame.
    new_price_data['Price_Rate_Of_Change'] = new_price_data.groupby('Symbol')['Close'].transform(lambda x: x.pct_change(periods = n))

    # Print the first 30 rows
    return new_price_data

In [86]:
rate_of_change(w,new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,...,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,...,0.00,1.70,100.000000,NaN,NaN,NaN,NaN,0.038141,0.021189,NaN
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,...,0.00,7.70,100.000000,NaN,NaN,NaN,NaN,0.287376,0.130282,NaN
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,...,0.00,6.00,100.000000,NaN,NaN,NaN,NaN,0.610317,0.292896,NaN
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,...,8.65,0.00,58.293541,NaN,NaN,NaN,NaN,0.428747,0.333308,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,...,0.00,4.25,73.223117,111.0,137.0,97.884615,-2.115385,-0.298741,-2.412779,0.100847
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,...,0.00,0.60,73.976523,111.0,140.7,87.710438,-12.289562,0.514220,-1.827379,0.097278
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,...,4.40,0.00,59.751127,111.0,140.7,72.895623,-27.104377,0.794298,-1.303044,0.073654
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,...,1.10,0.00,56.610918,111.0,140.7,69.191919,-30.808081,0.916931,-0.859049,0.048625


In [87]:
def obv(group):

    # Grab the volume and close column.
    volume = group['Volume']
    change = group['Close'].diff()

    # intialize the previous OBV
    prev_obv = 0
    obv_values = []

    # calculate the On Balance Volume
    for i, j in zip(change, volume):

        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        # OBV.append(current_OBV)
        prev_obv = current_obv
        obv_values.append(current_obv)
    
    # Return a panda series.
    return pd.Series(obv_values, index = group.index)
        
def apply_obv_to_groups(new_price_data):
# apply the function to each group
    obv_groups = new_price_data.groupby('Symbol').apply(obv)

    # add to the data frame, but drop the old index, before adding it.
    new_price_data['On Balance Volume'] = obv_groups.reset_index(level=0, drop=True)

    # display the data frame.
    return new_price_data

In [88]:
apply_obv_to_groups(new_price_data)

,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,...,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
1023,2012-01-17,ADANIPORTS,135.50,137.10,141.00,135.00,140.10,140.00,138.13,1636196,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0
1024,2012-01-18,ADANIPORTS,140.00,142.00,143.80,138.70,143.00,141.70,141.25,890591,...,1.70,100.000000,NaN,NaN,NaN,NaN,0.038141,0.021189,NaN,890591
1025,2012-01-19,ADANIPORTS,141.70,144.00,150.55,143.15,149.50,149.40,146.72,1456077,...,7.70,100.000000,NaN,NaN,NaN,NaN,0.287376,0.130282,NaN,2346668
1026,2012-01-20,ADANIPORTS,149.40,151.90,157.60,150.25,155.40,155.40,153.76,1634070,...,6.00,100.000000,NaN,NaN,NaN,NaN,0.610317,0.292896,NaN,3980738
1027,2012-01-23,ADANIPORTS,155.40,155.40,155.40,145.10,146.40,146.75,149.54,1657609,...,0.00,58.293541,NaN,NaN,NaN,NaN,0.428747,0.333308,NaN,2323129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,...,4.25,73.223117,111.0,137.0,97.884615,-2.115385,-0.298741,-2.412779,0.100847,75003139
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,...,0.60,73.976523,111.0,140.7,87.710438,-12.289562,0.514220,-1.827379,0.097278,76813885
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,...,0.00,59.751127,111.0,140.7,72.895623,-27.104377,0.794298,-1.303044,0.073654,75065086
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,...,0.00,56.610918,111.0,140.7,69.191919,-30.808081,0.916931,-0.859049,0.048625,73965373


In [92]:
def close_group(new_price_data):
    # Group by the `Symbol` column, then grab the `Close` column.
    close_groups = new_price_data.groupby('Symbol')['Close']
    # Apply the lambda function which will return -1.0 for down, 1.0 for up and 0.0 for no change.
    close_groups = close_groups.transform(lambda x : np.sign(x.diff()))
    new_price_data['Prediction'] = close_groups
    new_price_data.loc[new_price_data['Prediction'] == 0.0] = 1.0

In [93]:
close_group(new_price_data)

In [95]:
def close_group_nan_remove(new_price_data):
    # We need to remove all rows that have an NaN value.
    print('Before NaN Drop we have {} rows and {} columns'.format(new_price_data.shape[0], new_price_data.shape[1]))

    # Any row that has a `NaN` value will be dropped.
    new_price_data = new_price_data.dropna()

    # Display how much we have left now.
    print('After NaN Drop we have {} rows and {} columns'.format(new_price_data.shape[0], new_price_data.shape[1]))

    # Print the head.
    return new_price_data

In [96]:
close_group_nan_remove(new_price_data)

Before NaN Drop we have 3552 rows and 23 columns
After NaN Drop we have 3524 rows and 23 columns


,Date,Symbol,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,...,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Prediction
1037,2012-02-07,ADANIPORTS,148.10,149.15,149.45,137.85,137.85,139.15,142.49,1735290,...,39.886073,137.0,157.6,10.436893,-89.563107,-0.184165,0.053468,-0.006071,987669,-1.0
1038,2012-02-08,ADANIPORTS,139.15,139.00,143.30,136.60,140.00,140.30,140.87,1532614,...,42.123109,136.6,157.6,17.619048,-82.380952,-0.498153,-0.060051,-0.009880,2520283,1.0
1039,2012-02-09,ADANIPORTS,140.30,139.65,141.80,135.20,138.00,137.95,138.09,1907732,...,38.725216,135.2,157.6,12.276786,-87.723214,-0.876860,-0.227176,-0.076640,612551,-1.0
1040,2012-02-10,ADANIPORTS,137.95,138.15,144.15,137.20,142.95,142.90,140.28,1780060,...,48.769195,135.2,155.4,38.118812,-61.881188,-0.844026,-0.352810,-0.080438,2392611,1.0
1041,2012-02-13,ADANIPORTS,142.90,143.95,144.30,141.00,142.75,143.05,142.63,718239,...,49.061144,135.2,152.9,44.350282,-55.649718,-0.798905,-0.443333,-0.025213,3110850,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2012-01-10,MUNDRAPORT,132.20,133.85,137.00,132.85,136.30,136.45,135.67,869679,...,73.223117,111.0,137.0,97.884615,-2.115385,-0.298741,-2.412779,0.100847,75003139,1.0
1019,2012-01-11,MUNDRAPORT,136.45,136.00,140.70,134.75,135.75,137.05,137.28,1810746,...,73.976523,111.0,140.7,87.710438,-12.289562,0.514220,-1.827379,0.097278,76813885,1.0
1020,2012-01-12,MUNDRAPORT,137.05,137.50,137.50,131.20,131.60,132.65,133.23,1748799,...,59.751127,111.0,140.7,72.895623,-27.104377,0.794298,-1.303044,0.073654,75065086,-1.0
1021,2012-01-13,MUNDRAPORT,132.65,132.50,133.00,129.55,131.80,131.55,131.93,1099713,...,56.610918,111.0,140.7,69.191919,-30.808081,0.916931,-0.859049,0.048625,73965373,-1.0


Pre processing Completed


In [ ]:
# Grab our X & Y Columns.
X_Cols = price_data[['RSI','k_percent','r_percent',
                     'Price_Rate_Of_Change','MACD',
                     'On Balance Volume']]
Y_Cols = price_data['Prediction']

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state = 0)

# Create a Random Forest Classifier
rand_frst_clf = RandomForestClassifier(n_estimators = 100, oob_score = True, criterion = "gini", random_state = 0)

# Fit the data to the model
rand_frst_clf.fit(X_train, y_train)

# Make predictions
y_pred = rand_frst_clf.predict(X_test)

KeyError: "['RSI', 'r_percent', 'Price_Rate_Of_Change', 'MACD', 'On Balance Volume'] not in index"

In [ ]:
# Define the traget names
target_names = ['Down Day', 'Up Day']

# Build a classifcation report
report = classification_report(y_true = y_test, y_pred = y_pred, target_names = target_names, output_dict = True)

# Add it to a data frame, transpose it for readability.
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
Down Day,0.730479,0.700483,0.715166,414.00000
Up Day,0.709602,0.739024,0.724014,410.00000
accuracy,0.719660,0.719660,0.719660,0.71966
macro avg,0.720040,0.719754,0.719590,824.00000
weighted avg,0.720091,0.719660,0.719569,824.00000


In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

rf_matrix = confusion_matrix(y_test, y_pred)

true_negatives = rf_matrix[0][0]
false_negatives = rf_matrix[1][0]
true_positives = rf_matrix[1][1]
false_positives = rf_matrix[0][1]

accuracy = (true_negatives + true_positives) / (true_negatives + true_positives + false_negatives + false_positives)
percision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)

print('Accuracy: {}'.format(float(accuracy)))
print('Percision: {}'.format(float(percision)))
print('Recall: {}'.format(float(recall)))
print('Specificity: {}'.format(float(specificity)))

disp = plot_confusion_matrix(rand_frst_clf, X_test, y_test, display_labels = ['Down Day', 'Up Day'], normalize = 'true', cmap=plt.cm.Blues)
disp.ax_.set_title('Confusion Matrix - Normalized')
plt.show()

In [ ]:
# Calculate feature importance and store in pandas series
feature_imp = pd.Series(rand_frst_clf.feature_importances_, index=X_Cols.columns).sort_values(ascending=False)
feature_imp

In [ ]:
# store the values in a list to plot.
x_values = list(range(len(rand_frst_clf.feature_importances_)))

# Cumulative importances
cumulative_importances = np.cumsum(feature_imp.values)

# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')

# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin = 0, xmax = len(feature_imp), color = 'r', linestyles = 'dashed')

# Format x ticks and labels
plt.xticks(x_values, feature_imp.index, rotation = 'vertical')

# Axis labels and title
plt.xlabel('Variable')
plt.ylabel('Cumulative Importance')
plt.title('Random Forest: Feature Importance Graph')

In [ ]:
# Create an ROC Curve plot.
rfc_disp = plot_roc_curve(rand_frst_clf, X_test, y_test, alpha = 0.8)
plt.show()

In [ ]:
print('Random Forest Out-Of-Bag Error Score: {}'.format(rand_frst_clf.oob_score_))

In [ ]:
# Number of trees in random forest
# Number of trees is not a parameter that should be tuned, but just set large enough usually. There is no risk of overfitting in random forest with growing number of # trees, as they are trained independently from each other. 
n_estimators = list(range(200, 2000, 200))

# Number of features to consider at every split
max_features = ['auto', 'sqrt', None, 'log2']

# Maximum number of levels in tree
# Max depth is a parameter that most of the times should be set as high as possible, but possibly better performance can be achieved by setting it lower.
max_depth = list(range(10, 110, 10))
max_depth.append(None)

# Minimum number of samples required to split a node
# Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree. Too high values can also lead to # under-fitting hence depending on the level of underfitting or overfitting, you can tune the values for min_samples_split.
min_samples_split = [2, 5, 10, 20, 30, 40]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 7, 12, 14, 16 ,20]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# New Random Forest Classifier to house optimal parameters
rf = RandomForestClassifier()

# Specfiy the details of our Randomized Search
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
# With the new Random Classifier trained we can proceed to our regular steps, prediction.
rf_random.predict(X_test)


'''
    ACCURACY
'''
# Once the predictions have been made, then grab the accuracy score.
print('Correct Prediction (%): ', accuracy_score(y_test, rf_random.predict(X_test), normalize = True) * 100.0)


'''
    CLASSIFICATION REPORT
'''
# Define the traget names
target_names = ['Down Day', 'Up Day']

# Build a classifcation report
report = classification_report(y_true = y_test, y_pred = y_pred, target_names = target_names, output_dict = True)

# Add it to a data frame, transpose it for readability.
report_df = pd.DataFrame(report).transpose()
display(report_df)
print('\n')

'''
    FEATURE IMPORTANCE
'''
# Calculate feature importance and store in pandas series
feature_imp = pd.Series(rand_frst_clf.feature_importances_, index=X_Cols.columns).sort_values(ascending=False)
display(feature_imp)

In [ ]:
'''
    ROC CURVE
'''

fig, ax = plt.subplots()

# Create an ROC Curve plot.
rfc_disp = plot_roc_curve(rand_frst_clf, X_test, y_test, alpha = 0.8, name='ROC Curve', lw=1, ax=ax)

# Add our Chance Line
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

# Make it look pretty.
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="ROC Curve Random Forest")

# Add the legend to the plot
ax.legend(loc="lower right")

plt.show()

In [ ]:
print(y_test)

In [ ]:
print(X_test)